## Initialization

Mount Google Drive to load coda and data.

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


We change the directory to the drive folder with the colan tutorial files

In [10]:
# Change current path
# We may have to change this so it is universal
%cd "/content/"
! ls

/content
drive  sample_data


# Data

In [0]:
train_path = '/data/02-21.10way.clean'
valid_path = '/data/22.auto.clean'
test_path  = '/data/23.auto.clean'

Our data files consist of lines of the Penn Tree Bank data set. Each line is a sentence in a tree shape. Let's pretty print the first line from the training set to see what we're dealing with!

In [0]:
import os
from nltk import Tree
from nltk.treeprettyprinter import TreePrettyPrinter

def filereader(path):
    with open(path, mode='r') as f:
        for line in f:
            yield line

In [0]:
line = next(filereader(os.getcwd() + train_path))
tree = Tree.fromstring(line)
print(line)
print(TreePrettyPrinter(tree))

(TOP (S (PP (IN In) (NP (NP (DT an) (NNP Oct.) (CD 19) (NN review)) (PP (IN of) (NP (`` ``) (NP (DT The) (NN Misanthrope)) ('' '') (PP (IN at) (NP (NP (NNP Chicago) (POS 's)) (NNP Goodman) (NNP Theatre))))) (PRN (-LRB- -LRB-) (`` ``) (S (NP (JJ Revitalized) (NNS Classics)) (VP (VBP Take) (NP (DT the) (NNP Stage)) (PP (IN in) (NP (NNP Windy) (NNP City))))) (, ,) ('' '') (NP (NNP Leisure) (CC &) (NNP Arts)) (-RRB- -RRB-)))) (, ,) (NP (NP (NP (DT the) (NN role)) (PP (IN of) (NP (NNP Celimene)))) (, ,) (VP (VBN played) (PP (IN by) (NP (NNP Kim) (NNP Cattrall)))) (, ,)) (VP (VBD was) (VP (ADVP (RB mistakenly)) (VBN attributed) (PP (TO to) (NP (NNP Christina) (NNP Haag))))) (. .)))

                                                                                                                                                                TOP                                                                                                                                                       

We are, for now, just intereseted in the sentences, which is just the leaves of our tree

---



In [0]:
example_sentence = tree.leaves()
print(' '.join(example_sentence).lower())

in an oct. 19 review of `` the misanthrope '' at chicago 's goodman theatre -lrb- `` revitalized classics take the stage in windy city , '' leisure & arts -rrb- , the role of celimene , played by kim cattrall , was mistakenly attributed to christina haag .
